# KDDM1 - House Price Prediction

The goal of this project is to estimate the house price of different available houses.

The dataset consists of synthetic data, so it was generated by us. This allowed us to incorporate many properties different real world datasets have. Hence, please keep in mind that not each of the listed houses might be constructed like described in the real world (e.g. 20 sqm with 3 bathrooms), but still look out for other outliers in the data (like mentioned in the first lecture) and otherwise general logic still applies to this dataset (e.g. a house that was built thousands of years ago, but has the same properties as a newer one, may not make sense)!

Your tasks include 

- Exploratory Data Analysis and Preprocessing of the data
- Predicting the house prices
- Evaluating your model
- Answering the following Questions
  1. If I have a budget of €100,000, what kind of houses will I be able to buy?
  2. How much money can I save if I decide not to get a house with a pool?

You have to evaluate your model yourself, so consider what could be good and appropriate evaluation methods.
At the end of this semester, you have to give a short presentation on your work.
Your presentation should have exactly
- 1 or 2 EDA slides
- 1 Method slide
- 1 or 2 Evaluation slides
- 1 Discussion slide

Of course, you're allowed to add a title slide if you want to. The presentation should take a maximum of 10 minutes.
Please hand in your presentation (Group_xx.pdf) and your source code (Group_xx.zip) until 23.06.2024.
Have fun!

In [ ]:
%use dataframe
%use kandy
%use multik
%use kmath

In [ ]:
// We have 5 columns that are completely empty, we can ignore those immediatly
val data = DataFrame.readCSV("../data_7.csv").dropNulls(whereAllNull = true)
data

## Feature Description
- Bedrooms - number of bedrooms
- Bathrooms - number of bathrooms
- SquareFootageHouse - the square footage of the house in square meters
- Location - whether the house is urban, rural or suburban
- Age - the age of the house, all houses were built in the last 100 years
- PoolQuality - the quality of the pool, if it is 'None' then there is no pool
- HasPhotovoltaics - if the house has photovoltaics
- HeatingType - the heating type of the house
- HasFiberglass - if the house has the possibility of a fiberglass connection
- IsFurnished - if the home is furnished
- DateSinceForSale - the date the house was put on the market
- HouseColor - the color of the house
- PreviousOwnerName - the first name of the previous owner
- HasFireplace - presence of a fireplace
- KitchensQuality - quality of the kitchen(s)
- BathroomsQuality - quality of bathroom(s)
- BedroomsQuality - quality of the bedroom(s)
- LivingRoomsQuality - quality of the living room(s)
- SquareFootageGarden - the square footage of the garden in square meters
- PreviousOwnerRating - the rating of the house by the previous owner
- HeatingCosts - the heating costs in Euro per month for the whole house
- Window_model_names - the name of the window model
- Price - the price of the house in thousands of euros 

In [ ]:
data.describe()

In [ ]:
import java.time.Year

val ownerNames = listOf(
    "Stefan", "Roman", "Natalie", "Denis", "Maria", "Florian", "Laura",
    "Theresa", "Johanna", "Martin", "Daniel", "Sabrina", "Lukas", "Andreas", "Sandra", null,
)

// Check that the data looks as we expect it to look. Further this step is done as an investigation in the data set, where we can see whether the data seems to be "ok" or not.
require(data.Bedrooms.all { it in listOf(1.0, 2.0, 3.0, 4.0, 5.0, null) })
require(data.Bathrooms.all { it in listOf(1.0, 2.0, 3.0, 4.0, 5.0, null) })
require(data.SquareFootageHouse.all { it in -895.0..8578.0 && it.toInt().toDouble() == it })
require(data.Location.all { it in listOf("Suburbann", "Urban", "Rural", "Suburban", null) })
require(data.Age.all { it == null || (it in -94.0..71.0 && it.toInt().toDouble() == it) })
require(data.PoolQuality.all { it in listOf("None", "Excellent", "Good", "Poor", null) })
require(data.HasPhotovoltaics.all { it in listOf(true, false, null) })
require(data.HeatingType.all { it in listOf("Gas", "Electric", "Electricity", "Oil Heating", "Oil") })
require(data.HasFiberglass.all { it in listOf(true, false) })
require(data.IsFurnished.all { it in listOf(true, false) })
require(data.DateSinceForSale.all { it.year in 2020..2024 })
require(data.HouseColor.all { it in listOf("Yellow", "Green", "Gray", "White", null) })
require(data.PreviousOwnerName.all { it in ownerNames })
require(data.HasFireplace.all { it in listOf(true, false, null) })
require(data.KitchensQuality.all { it in listOf("Excellent", "Good", "Poor") })
require(data.BathroomsQuality.all { it in listOf("Excellent", "Good", "Poor") })
require(data.BedroomsQuality.all { it in listOf("Excellent", "Good", "Poor") })
require(data.LivingRoomsQuality.all { it in listOf("Excellent", "Good", "Poor") })
require(data.SquareFootageGarden.all { it in listOf(6.0, 13.0, 18.0, 30.0) })
require(data.PreviousOwnerRating.all { it in 1.0..4.97 }) // is real double
require(data.HeatingCosts.all { it == null || it in -443.0..8292.0 }) // is real double
require(data.WindowModelNames.all { it.matches(Regex("(?:.*?Wood.*?)|(?:.*?Aluminum.*?)|(?:.*?Steel.*?)")) })
require(data.Price.all { it in 29.12382..6050.5916 }) // is real double

// We create filters, such that we can see (by numbers) how the data looks (how many entries seem to be wrong). We defined different ranges based on the decision you can see after the filter (all squarefootages are, per the assignment, considered to be in m^2)
val numValidSquareFootages = data.SquareFootageHouse.count { it in 17.0..151.0 }        // The smallest squarefootage we were able to find, that is positive is 17, the highest squarefootage that seemed to be reasonable was 151 )
val numTooSmallSquareFootages = data.SquareFootageHouse.count { it < 17.0 }             // The number of too small houses (e.g. negative squarefootage)
val numTooLargeSquareFootages = data.SquareFootageHouse.count { it > 151.0 }            // The number of unreasonable houses (e.g. squarefootage well above 151, the data set reflects houses not warehouses)
println("Valid square footage values: $numValidSquareFootages")
println("Too small square footage values: $numTooSmallSquareFootages")
println("Too large square footage values: $numTooLargeSquareFootages\n")

val numValidAges = data.Age.count { it != null && it >= 0.0 }                           // Per the assignment all houses have an age between 0 to 100 years, everything else is not correct
val numInvalidAges = data.Age.count { it != null && it < 0.0 }                          // Houses with negative age are not relevant to this assignment (although they could be considered housese "under construction", where the negative number reflects the time it will take until the house is finished)
println("Valid age values: $numValidAges")
println("Invalid age values: $numInvalidAges\n")

val numValidHeatingCosts = data.HeatingCosts.count { it != null && it in 12.6..238.4 }  // Heating costs that are bellow hour threashold are all negative, this value has been chosen as it is the smallest _positive_ heating cost. Heating cost that are too high are flagged as they are likely outliers
val numTooSmallHeatingCosts = data.HeatingCosts.count { it != null && it < 12.6 }       // We omitted the negative heating cost, we discussed the possibility of the heating cost being negative iff the heating type is electric and the house has photovoltaic and we assume that the photovoltaic system delivers enough energy to heat the house. 
                                                                                        // We have omitted this approach as we decided that the _heating cost_ would still apply. 
val numTooLargeHeatingCosts = data.HeatingCosts.count { it != null && it > 238.4 }  
println("Valid heating cost values: $numValidHeatingCosts")
println("Too small heating cost values: $numTooSmallHeatingCosts")
println("Too large heating cost values: $numTooLargeHeatingCosts\n")

val potentiallyInvalidPriceValues = data.Price.filter { it > 1000.0 }                   // We flagged priced that seemed unreasonably high (e.g. 1,000€ in thousand of € would translate to 1,000,000€)
println("Maybe wrong price values: $potentiallyInvalidPriceValues\n")

println("Remember to argue this:")
// Houses that have been on sale for quite some time (e.g. 2023-01-16) can have an age of 0.0 years since the house could be for sale for longer than it has been finished, making the age of the house 0.0 but the date since for sale significantly higher (e.g. Kohlbacher sells a certain amount of houses before they start building them)
println(data.filter {
    val age = it.Age
    age != null && age >= 0.0 && age < 5.0
}["Age", "DateSinceForSale"])

## Simple Preprocessing description

For this jupyter notebook it was necessary to translate the categorical data (e.g. house color) to a numerical representation. To do this, we used enums, such that we can later easily get back to the original value.

In [ ]:
// Define enums

enum class LocationType {
    SUBURBAN,
    URBAN,
    RURAL,
}

enum class NullableQuality {
    NONE,
    POOR,
    GOOD,
    EXCELLENT,
}

enum class Quality(val stringName: String) {
    POOR("Poor"),
    GOOD("Good"),
    EXCELLENT("Excellent"),
}

enum class HeatingTypeEnum {
    GAS,
    ELECTRIC,
    OIL,
}

enum class HouseColorEnum {
    YELLOW,
    GREEN,
    GRAY,
    WHITE,
}

enum class WindowMaterialType {
    WOOD,
    ALUMINUM,
    STEEL,
}

fun qualityLookup(value: String): Int {
    return Quality.values().first { it.stringName == value }.ordinal
}

// Apply simple pre-processing by removing invalid data and fixing typos
// Further we checked all columns whether it is possible to set the column type to _int_. If that is the case we did that
val simplePreprocessedData = data
    .replace(data.Bedrooms).with { column ->
        column.convertToInt()
    }
    .replace(data.Bathrooms).with { column ->
        column.convertToInt()
    }
    .replace(data.SquareFootageHouse).with { column ->
        column.convertToInt().map {
            when (it) {
                in 17..151 -> it
                in 0..17 -> error("SquareFootageHouse contains unexpected value")
                in 151..1000 -> error("SquareFootageHouse contains unexpected value")
                // We are imputing outliers to missing values already here, but this is fine,
                // we have tried other methods and received similar results. Additionally,
                // these values are so too little to change the correlation by much.
                else -> null
            }
        }
    }
    .replace(data.Location).with { column ->
        column.map {
            when (it) {
                // We assumend a type for "Suburbann" and updated it to "Suburban" as this seemed like a user-input error
                "Suburbann", "Suburban" -> LocationType.SUBURBAN
                "Urban" -> LocationType.URBAN
                "Rural" -> LocationType.RURAL
                null -> null
                else -> error("Location contains unexpected value")
            }?.ordinal
        }
    }
    .replace(data.Age).with { column ->
        column.convertToInt().map {
            when {
                // We are imputing outliers to missing values already here, but this is fine,
                // we have tried other methods and received similar results. Additionally,
                // these values are too little to change the correlation by much.
                it == null || it < 0 -> null
                it <= 71 -> it
                else -> error("Age contains unexpected value")
            }
        }
    }
    .replace(data.PoolQuality).with { column ->
        column.map {
            when (it) {
                null -> null
                // Just set the enum values instead of the types, such that we can later on correlate the data
                "None" -> NullableQuality.NONE
                "Poor" -> NullableQuality.POOR
                "Good" -> NullableQuality.GOOD
                "Excellent" -> NullableQuality.EXCELLENT
                else -> error("PoolQuality contains unexpected value")
            }?.ordinal
        }
    }
    .replace(data.HasPhotovoltaics).with { column ->
        column.map {
            when (it) {
                // We switch the textual representation of boolean to integer representation such that we can correlate the data
                false -> 0
                true -> 1
                null -> null
            }
        }
    }
    .replace(data.HeatingType).with { column ->
        column.map {
            when (it) {
                // Just set the enum values instead of the types, such that we can later on correlate the data
                // We further updated the "Electricity" and "Oil Heating" to "Electric" and "Oil" respectively 
                // as they are the same and just highlight inconsistencies in data allocation
                "Gas" -> HeatingTypeEnum.GAS
                "Electric", "Electricity" -> HeatingTypeEnum.ELECTRIC
                "Oil Heating", "Oil" -> HeatingTypeEnum.OIL
                else -> error("HeatingType contains unexpected value")
            }.ordinal
        }
    }
    .replace(data.HasFiberglass).with { column ->
        column.map {
            when (it) {
                false -> 0
                true -> 1
            }
        }
    }
    .replace(data.IsFurnished).with { column ->
        column.map {
            when (it) {
                false -> 0
                true -> 1
            }
        }
    }
    .replace(data.DateSinceForSale).with { column ->
        column.map {
            it.toEpochDays()
        }
    }
    .replace(data.HouseColor).with { column ->
        column.map {
            when (it) {
                "Yellow" -> HouseColorEnum.YELLOW
                "Green" -> HouseColorEnum.GREEN
                "Gray" -> HouseColorEnum.GRAY
                "White" -> HouseColorEnum.WHITE
                null -> null
                else -> error("HouseColor contains unexpected value")
            }?.ordinal
        }
    }
    .remove(data.PreviousOwnerName) // We can not ensure that same name means same person, so this is useless
    .replace(data.HasFireplace).with { column ->
        column.map {
            when (it) {
                // We switch the textual representation of boolean to integer representation such that we can correlate the data
                false -> 0
                true -> 1
            }
        }
    }
    // The next few lines all switch the "Quality" attributes to a numerical represenation
    .replace(data.KitchensQuality).with {
        it.map(transform = ::qualityLookup)
    }
    .replace(data.BathroomsQuality).with {
        it.map(transform = ::qualityLookup)
    }
    .replace(data.BedroomsQuality).with {
        it.map(transform = ::qualityLookup)
    }
    .replace(data.LivingRoomsQuality).with {
        it.map(transform = ::qualityLookup)
    }
    // Convert the SquareFootageGarden to integer as there are no double values
    .replace(data.SquareFootageGarden).with { column ->
        column.convertToInt()
    }
    // .replace(data.PreviousOwnerRating) -> nothing to do
    .replace(data.HeatingCosts).with { column ->
        column.map {
            when (it) {
                null -> null
                in 12.6..238.4 -> it
                in 0.0..12.5, in 238.5..5000.0 -> error("HeatingCosts contains unexpected value")
                // We are imputing invalid values with missing values already here, but this is fine,
                // have tried other methods and received similar results. Additionally, these values
                // are so too little to change the correlation by much.
                else -> null
            }
        }
    }
    .replace(data.WindowModelNames).with { column ->
        // When looking at the dataset, we found that all window brand names are unique. However, we found that in all window brand names the type of material is statet. We checked the brand name for all  possible materials and imputed those values.
        column.map {
            when {
                "Wood" in it -> WindowMaterialType.WOOD
                "Aluminum" in it -> WindowMaterialType.ALUMINUM
                "Steel" in it -> WindowMaterialType.STEEL
                else -> error("WindowModelNames contains unexpected value")
            }.ordinal
        }
    }
// .replace(data.Price) -> nothing to do
simplePreprocessedData.writeCSV("simple_preprocessed_data_7.csv")
simplePreprocessedData.describe()

                // We switch the textual representation of boolean to integer representation such that we can correlate the data
## Correlation

TODO: Describe, ran in python

| First Column |    Second Column    |   MIC    |
|:------------:|:-------------------:|:--------:|
| PoolQuality  | SquareFootageGarden | 0.852385 |
| HeatingCosts | SquareFootageHouse  | 0.341288 |
|   Location   |        Price        | 0.321912 |
| HeatingCosts |     HeatingType     | 0.294607 |
|    Price     | SquareFootageHouse  | 0.277127 |
|  HouseColor  | PreviousOwnerRating | 0.230396 |
|  Bathrooms   | SquareFootageHouse  | 0.223996 |
|   Bedrooms   | SquareFootageHouse  | 0.221773 |
| HeatingCosts |        Price        | 0.178266 |
| HeatingType  |        Price        | 0.150037 |

| First Column |    Second Column    |  Pearson  |
|:------------:|:-------------------:|:---------:|
| PoolQuality  | SquareFootageGarden | 0.986398  |
| HeatingCosts | SquareFootageHouse  | 0.687899  |
|  Bathrooms   | SquareFootageHouse  | 0.591070  |
|   Bedrooms   | SquareFootageHouse  | 0.572745  |
|  Bathrooms   |      Bedrooms       | 0.425090  |
| HeatingCosts |     HeatingType     | 0.421382  |
|   Bedrooms   |    HeatingCosts     | 0.404967  |
|  Bathrooms   |    HeatingCosts     | 0.332536  |
|  HouseColor  | PreviousOwnerRating | -0.340979 |


| First Column |    Second Column    |  Kendall  |
|:------------:|:-------------------:|:---------:|
| PoolQuality  | SquareFootageGarden | 1.000000  |
| HeatingCosts | SquareFootageHouse  | 0.568447  |
|   Location   |        Price        | 0.535824  |
|  Bathrooms   | SquareFootageHouse  | 0.492957  |
|   Bedrooms   | SquareFootageHouse  | 0.477654  |
| HeatingCosts |        Price        | 0.367662  |
|   Bedrooms   |    HeatingCosts     | 0.356424  |
|  Bathrooms   |      Bedrooms       | 0.332152  |
|    Price     | SquareFootageHouse  | 0.325567  |
|  Bathrooms   |    HeatingCosts     | 0.292921  |
| HeatingCosts |     HeatingType     | 0.275083  |


| First Column |    Second Column    | Spearman  |
|:------------:|:-------------------:|:---------:|
| PoolQuality  | SquareFootageGarden | 1.000000  |
| HeatingCosts | SquareFootageHouse  | 0.731819  |
|   Location   |        Price        | 0.668827  |
|  Bathrooms   | SquareFootageHouse  | 0.599644  |
|   Bedrooms   | SquareFootageHouse  | 0.576847  |
| HeatingCosts |        Price        | 0.524518  |
|    Price     | SquareFootageHouse  | 0.470928  |
|   Bedrooms   |    HeatingCosts     | 0.446096  |
|  HouseColor  | PreviousOwnerRating | -0.288551 |


## Imputation

The following cell contains the actual imputations of the values.


In [42]:
import Line_15_jupyter.*
import org.apache.commons.math3.distribution.NormalDistribution

/*
Column            | Type      | # nulls |          | Type of imputement
---------------------------------------------------------------------------------------------------------------------
HeatingCosts        Double?     487     Done        MAR: Correlated from heating type and square footage house
Bedrooms            Int?        377     Done        MAR: Correlated to square footage house
Bathrooms           Int?        365     Done        MAR: Correlated to square footage house
Location            Int?        194     Done        MAR: Correlated to price
Age                 Int?        139     Done        MCAR/NMAR: House not built yet (?); High age not reported (?)
PoolQuality         Int?        119     Done        MAR: Directly related to square footage garden (buckets)
HasPhotovoltaics    Int?        79      Done        MCAR/NMAR: People with no photovoltaics ignore this (?)
SquareFootageHouse  Int?        22      Done        MAR: Heating Type and Cost is correlated to the SFH
HouseColor          Int?        8       Done        MAR: Rating is correlated to the color
 */

val random = Random(0L)


// Calculate the mean rating for each color
val meanRatingValueToColorOrdinal: Map<Double, Int> = HouseColorEnum.values().associate { color ->
    val meanRating = simplePreprocessedData
        .filter { it.HouseColor == color.ordinal }
        .mean("PreviousOwnerRating")
    meanRating to color.ordinal
}

// line = intercept to k
// When looking at the data we found, that heating cost and the SquareFootageHouse is linearly correlated, with the HeatingType being the difference between the three regressions.
val heatingTypeToLinearRegressionLine: Map<Int, Pair<Double, Double>> =
    mapOf(
        HeatingTypeEnum.GAS.ordinal to (1.98838 to 1.00022),
        HeatingTypeEnum.ELECTRIC.ordinal to (-0.054492 to 1.65962),
        HeatingTypeEnum.OIL.ordinal to (-0.487318 to 0.625142),
    )

val hasPhotovoltaicsDistribution = simplePreprocessedData.HasPhotovoltaics.mean()

val ageNormalDistribution = NormalDistribution(simplePreprocessedData.Age.mean(), simplePreprocessedData.Age.std())

val meanPriceValueToLocation: Map<Double, Int> = LocationType.values().associate { location ->
    val meanPrice = simplePreprocessedData
        .filter { it.Location == location.ordinal }
        .mean("Price")
    meanPrice to location.ordinal
}

val meanSquareFootageHouseValueToBathrooms: Map<Double, Int> = (1..5).toList().associate { numBathrooms ->
    val meanSquareFootageHouse = simplePreprocessedData
        .filter { it.Bathrooms == numBathrooms }
        .mean("SquareFootageHouse")
    meanSquareFootageHouse to numBathrooms
}

val meanSquareFootageHouseValueToBedrooms: Map<Double, Int> = (1..5).toList().associate { numBedrooms ->
    val meanSquareFootageHouse = simplePreprocessedData
        .filter { it.Bedrooms == numBedrooms }
        .mean("SquareFootageHouse")
    meanSquareFootageHouse to numBedrooms
}

fun getLikliestHouseColorFromRating(actualRating: Double): Int {
    val minMeanRatingToDiff = meanRatingValueToColorOrdinal.keys
        .map { it to abs(it - actualRating) }
        .minBy { it.second }
    return meanRatingValueToColorOrdinal[minMeanRatingToDiff.first]!!
}

fun interpolateSquareFootageHouseFromHeatingTypeAndCost(heatingTypeOrdinal: Int, heatingCost: Double?): Int? {
    if (heatingCost == null) {
        return null
    }
    val linearRegressionLine = heatingTypeToLinearRegressionLine[heatingTypeOrdinal]!!
    return (linearRegressionLine.first + linearRegressionLine.second * heatingCost).roundToInt()
}

fun imputeHasPhotovoltaics(): Int {
    return when (random.nextDouble() <= hasPhotovoltaicsDistribution) {
        false -> 0
        true -> 1
    }
}

fun getLikliestPoolQualityFromSquareFootageGarden(squareFootageGarden: Int): Int {
    return when (squareFootageGarden) {
        6 -> NullableQuality.NONE
        13 -> NullableQuality.POOR
        18 -> NullableQuality.GOOD
        30 -> NullableQuality.EXCELLENT
        else -> error("Invalid square footage found: $squareFootageGarden")
    }.ordinal
}

fun imputeAge(): Int {
    return ageNormalDistribution.sample().roundToInt()
}

fun getLikliestLocationFromPrice(actualPrice: Double): Int {
    val minMeanPriceToDiff = meanPriceValueToLocation.keys
        .map { it to abs(it - actualPrice) }
        .minBy { it.second }
    return meanPriceValueToLocation[minMeanPriceToDiff.first]!!
}

fun getLikliestNumBathroomsFromSquareFootageHouse(actualSquareFootageHouse: Int?): Int? {
    if (actualSquareFootageHouse == null) return null
    val minMeanSquareFootageHouseToDiff = meanSquareFootageHouseValueToBathrooms.keys
        .map { it to abs(it - actualSquareFootageHouse) }
        .minBy { it.second }
    return meanSquareFootageHouseValueToBathrooms[minMeanSquareFootageHouseToDiff.first]!!
}

fun getLikliestNumBedroomsFromSquareFootageHouse(actualSquareFootageHouse: Int?): Int? {
    if (actualSquareFootageHouse == null) return null
    val minMeanSquareFootageHouseToDiff = meanSquareFootageHouseValueToBedrooms.keys
        .map { it to abs(it - actualSquareFootageHouse) }
        .minBy { it.second }
    return meanSquareFootageHouseValueToBedrooms[minMeanSquareFootageHouseToDiff.first]!!
}

fun interpolateHeatingCostFromHeatingTypeAndSquareFootageHouse(
    heatingTypeOrdinal: Int,
    squareFootageHouse: Int?,
): Double? {
    if (squareFootageHouse == null) {
        return null
    }
    val linearRegressionLine = heatingTypeToLinearRegressionLine[heatingTypeOrdinal]!!
    return (squareFootageHouse - linearRegressionLine.first) / linearRegressionLine.second
}

// ----------------------------------------------------------------------------
// MAR
// We impute the SquareFootageHouse by using the linear regression for SquareFootageHouse, HeatingCost and HeatingType.
// With this we were able to impute halve the values that were missing, the other halve we filtered and removed, they were not to be recovered with useful means.
val dataWithImputedSquareFootageHouse =
    simplePreprocessedData.replace(simplePreprocessedData.SquareFootageHouse).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else interpolateSquareFootageHouseFromHeatingTypeAndCost(
                simplePreprocessedData.HeatingType[index],
                simplePreprocessedData.HeatingCosts[index],
            )
        }
    }.filter { it["SquareFootageHouse"] != null }

val imputedData = dataWithImputedSquareFootageHouse
    // MAR
    // We impute the color of the house by taking the MeanRating of each color and checking to which MeanRating the current rating has the least distance, this is the most likely color.
    .replace(dataWithImputedSquareFootageHouse.HouseColor).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value else getLikliestHouseColorFromRating(dataWithImputedSquareFootageHouse.PreviousOwnerRating[index])
        }
    }
    // MCAR
    // We took the distribution and imputet the values accordingly to not change the data, but we can not drop all of those rows
    .replace(dataWithImputedSquareFootageHouse.HasPhotovoltaics).with { column ->
        column.map { value ->
            if (value != null) value
            else imputeHasPhotovoltaics()
        }
    }
    // MAR
    // The bigger the garden, the better the pool. This can be seen when looking at the data, The pool quality directly correlates with the SquareFootageGarden.
    .replace(dataWithImputedSquareFootageHouse.PoolQuality).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else getLikliestPoolQualityFromSquareFootageGarden(dataWithImputedSquareFootageHouse.SquareFootageGarden[index])
        }
    }
    // MCAR
    // We impute with repsect to the distribution of the values such that we do not skew the data
    .replace(dataWithImputedSquareFootageHouse.Age).with { column ->
        column.map { value ->
            if (value != null) value
            else imputeAge()
        }
    }
    // MAR
    // Same as with the rating. We impute the location of the house by matching to the closest MeanSquarfootageHouse by Location
    .replace(dataWithImputedSquareFootageHouse.Location).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else getLikliestLocationFromPrice(dataWithImputedSquareFootageHouse.Price[index])
        }
    }
    // MAR
    // The number of bathrooms correlates with the SuareFootageHouse. There was a correlation between Bathrooms  and Bedrooms as well, though this was likely due to the fact that Bedrooms as well correlates to SquareFootageHouse.
    // This means, the bigger the house, the more bathrooms and bedrooms there are.
    // However, since there were houses with e.g. bedroom: 1, bathroom: 2, squarefootage: 20 and bedroom 2, bathroom: 1, squarefootage: 20 and bedroom: 2, bathroom: 2, squarefootage: 20 we moved on from the idea that the sum of bedrooms and bathrooms for a given size of house has to be a certain amount (e.g. 3=2 bedrooms + 1 bathroom)
    .replace(dataWithImputedSquareFootageHouse.Bathrooms).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else getLikliestNumBathroomsFromSquareFootageHouse(dataWithImputedSquareFootageHouse.SquareFootageHouse[index])
        }
    }
    // MAR
    // Look at answer above
    .replace(dataWithImputedSquareFootageHouse.Bedrooms).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else getLikliestNumBedroomsFromSquareFootageHouse(dataWithImputedSquareFootageHouse.SquareFootageHouse[index])
        }
    }
    // MAR
    // We replaced the missing SquareFootageHouse values with the LinearRegression we found between HeatingType, HeatingCost and SquareFootageHouse
    .replace(dataWithImputedSquareFootageHouse.HeatingCosts).with { column ->
        column.mapIndexed { index, value ->
            if (value != null) value
            else interpolateHeatingCostFromHeatingTypeAndSquareFootageHouse(
                dataWithImputedSquareFootageHouse.HeatingType[index],
                dataWithImputedSquareFootageHouse.SquareFootageHouse[index],
            )
        }
    }

imputedData.writeCSV("imputed_data_7.csv")
imputedData.describe()

name,type,count,unique,nulls,top,freq,mean,std,min,median,max
Bedrooms,Int,984,5,0,1,541,"1,758130","1,053230",1,1,5
Bathrooms,Int,984,5,0,1,523,"1,801829","1,090212",1,1,5
SquareFootageHouse,Int,984,134,0,103,13,"84,199187","36,835989",17,86,151
Location,Int,984,3,0,0,359,"0,949187","0,822781",0,1,2
Age,Int,984,76,0,45,35,"44,518293","16,362491",0,46,81
PoolQuality,Int,984,4,0,0,595,"0,801829","1,086473",0,0,3
HasPhotovoltaics,Int,984,2,0,0,775,"0,212398","0,409213",0,0,1
HeatingType,Int,984,3,0,2,390,"1,100610","0,826223",0,1,2
HasFiberglass,Int,984,2,0,0,599,"0,391260","0,488281",0,0,1
IsFurnished,Int,984,2,0,1,696,"0,707317","0,455225",0,1,1
